In [2]:
# Configure Environment 
from  dev.data.settings import * 
print("Platform: " , PLATFORM)
print("Data directory : ", os.getcwd())



Platform:  dev
Data directory :  /home/jovyan/dev/data


## Imports:

In [66]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from sklearn import preprocessing
import datetime
import pickle
import json

from IPython.display import HTML
import logging
logging.getLogger('tensorflow').disabled = True #OPTIONAL - to disable outputs from Tensorflow

## Create sentence embeddings from Elmo via TensorFlow Hub
https://tfhub.dev/google/elmo/1

In [67]:
import tensorflow.compat.v1 as tf
import tensorflow_hub as hub

url = "https://tfhub.dev/google/elmo/2"
embed = hub.Module(url)

## Read Plot Sentences

In [82]:
# Change Directory to Data
# read from data
# Create elmoWeights Folder, in not exists
# Write weights to data/emloWeights


# datadir = os.path.join(rootdir, 'data') 
path_to_json = './person/wikipedia/'

elmodir = os.getcwd()+'/models/'

# create elmoWeights directory - if it does not exist
if not os.path.exists(elmodir):
    os.makedirs(elmodir) 

## READ PLOT DATA - ALL from PKL file

In [73]:

json_files = [path_to_json+pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json') and os.stat(path_to_json+pos_json).st_size>0]
len(json_files)


bio_dict = {}
for f in json_files:
    data = json.load(open(f) )
    bio = data["biography"]
    bio_dict[data["_id"]] = bio
    


In [74]:
sentencesAll = list(bio_dict.values())
sentencesAll[0]

'Danai Jekesai Gurira (; born February 14, 1978) is an American actress and playwright. She is best known for her starring roles as Michonne on the AMC horror drama series The Walking Dead (2012–present) and as Okoye in the Marvel Cinematic Universe superhero films Black Panther (2018), Avengers: Infinity War (2018), and Avengers: Endgame (2019).\nGurira is also the playwright of the Broadway play Eclipsed, for which she was nominated for a Tony Award for Best Play.'

In [75]:
def cleanSentences(sentences):
    punctuation = '!"#$%&()*+-/:;<=>?@[\\]^_`{|}~'  #TBD
        #TBD - find sentence splitter - 
        # remove punctuation, strip white space; remove non-standard characters; remove numbers; 
    i = 0
    for s in sentences:
        s = s.strip()
        s = s.lower().replace('\n', ' ').replace('\t', ' ').replace('\xa0',' ').replace("[0-9]", " ")
        s = s.replace(',', '').replace('.','') #TBD - replace with punctuation
        s = s[0:4000]
        sentences[i] = s
        i += 1
    return sentences

In [76]:

# For Pickle Input
sentences = cleanSentences(sentencesAll)

In [78]:
# Test - 
sentences[0]
# len(sentences)

'danai jekesai gurira (; born february 14 1978) is an american actress and playwright she is best known for her starring roles as michonne on the amc horror drama series the walking dead (2012–present) and as okoye in the marvel cinematic universe superhero films black panther (2018) avengers: infinity war (2018) and avengers: endgame (2019) gurira is also the playwright of the broadway play eclipsed for which she was nominated for a tony award for best play'

# Score Sentences in Batches

In [79]:
# score n Sentences at time - to pipeline this
def defineEmbeddings (start, end):    
    embeddings = embed(
        sentences[start:end],
        signature="default",
        as_dict=True)["default"]
    return embeddings

In [80]:
# score sentences
def scoreSentences (embeddings):
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.tables_initializer())
        x = sess.run(embeddings)
    return x

In [83]:
# Save Weights - use names based on Movie 
import pickle as pkl

def saveWeights(elmoFN, x):
    with open('./models/'+elmoFN,'wb') as f: pkl.dump(x, f)
    return

## Score N sentences at a time


In [84]:
%matplotlib inline

from IPython.core.magic import register_line_magic


In [85]:
import math 

batch_size = 100                      # Specify number of sentences per file  (was 500)
numPlots = len(sentences)
numfiles = math.ceil(numPlots/batch_size) 
[os.remove(os.path.join(elmodir,f)) for f in os.listdir(elmodir) if 'elmoBioweights' in f]

start = 0
end = batch_size
for file in range(1,numfiles+1):
    elmoFN = 'elmoBioweights' + str(start) + 'to' + str(end) + '.pkl'
    print('Score sentences:', elmoFN)

        # Define Embeddings, Score, and Store Weights one Batch of N at time
    embeddings = defineEmbeddings(start, end)
    x = scoreSentences(embeddings)
    saveWeights(elmoFN,x)
    start = end
    end = end+batch_size

Score sentences: elmoBioweights0to100.pkl


## Read and Concatenate all weight arrays in elmoWeights to build


In [86]:
a='elmoPlotsweights250to300'
if 'Plot' in a:
    print('pass')

pass


In [87]:
# Read Weights - use names based on actorID 
import pickle as pkl
import numpy as np
import glob

def readElmoWeightFile(file):
    with open(elmodir+file, 'rb') as f:
        elmo_file = pkl.load(f)
    return list(elmo_file)
  
def readWeights(elmodir, modelname):
    noFiles = len([name for name in os.listdir(elmodir) if modelname in name ])
    xin = readElmoWeightFile('elmoBioweights0to'+str(batch_size)+'.pkl')
    size = batch_size
    for x in range(1, noFiles):
        start = x*size
        end = start+size
        filename = 'elmoBioweights' + str(start) + 'to' + str(end) + '.pkl'
        print(filename)
        xin += readElmoWeightFile(filename)
    return xin

# Save model and labels catalog.

In [88]:
os.listdir(elmodir)

['elmoBioweights0to100.pkl']

In [94]:
!ls

cast  models  movies  person  top100Actor.txt


In [91]:

xin = readWeights(elmodir, 'Bio')
print("Elmo Vector Shape {0}, {1}".format( len(xin), len(xin[1])) )

pickle.dump( bio_dict, open( "./models/bio_dict.pkl", "wb" ) )
pickle.dump( xin, open( "./models/elmoBioweights.pkl", "wb" ) )


Elmo Vector Shape 99, 1024


In [92]:
# Load models 

bio_dict = pickle.load( open( './models/bio_dict.pkl', 'rb' ) )
xin = pickle.load( open( './models/elmoBioweights.pkl', 'rb' ) )


In [95]:


topActors = pd.read_csv('./top100Actors.txt', 
                     sep='\t',
                     header=0, 
                    dtype=str)

actorsDF = topActors[['Rank','First Name','Last Name']]
actorsDF['Full Name']= actorsDF['First Name']+' '+actorsDF['Last Name']
actorsDF = actorsDF.drop(['First Name','Last Name'], axis=1)

celebrityLookup={}
for i, row in actorsDF.iterrows():
    celebrityLookup.update({ row['Rank'] : row['Full Name']})

celebrityLookup['10']

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


'Harrison Ford'

In [96]:
# Load models
pickle.dump( celebrityLookup, open( "./models/celebrityLookup.pkl", "wb" ) )
celebrityLookup = pickle.load( open( './models/celebrityLookup.pkl', 'rb' ) )

In [101]:
# Copy models upto S3
bucket = os.popen('aws ssm get-parameter --name jupyterBucketName-' + PLATFORM + ' --with-decryption | jq -r .Parameter.Value').read()
print(bucket)
executable = 'aws s3 sync ./models/ s3://' + bucket.strip() + '/models/ '
print(executable)
myCmd = os.popen(executable).read()
print(myCmd)

bdso-celebrity-content-dev-vpc-05447d97de18e28f8

aws s3 sync ./models/ s3://bdso-celebrity-content-dev-vpc-05447d97de18e28f8/models/ 
Completed 149.8 KiB/~9.9 MiB (298.8 KiB/s) with ~9 file(s) remaining (calculating...)
upload: models/bio_dict.pkl to s3://bdso-celebrity-content-dev-vpc-05447d97de18e28f8/models/bio_dict.pkl
Completed 149.8 KiB/~10.0 MiB (298.8 KiB/s) with ~9 file(s) remaining (calculating...)
Completed 349.9 KiB/10.1 MiB (536.7 KiB/s) with 10 file(s) remaining                  
Completed 352.7 KiB/10.1 MiB (538.0 KiB/s) with 10 file(s) remaining                  
upload: models/celebrityLookup.pkl to s3://bdso-celebrity-content-dev-vpc-05447d97de18e28f8/models/celebrityLookup.pkl
Completed 352.7 KiB/10.1 MiB (538.0 KiB/s) with 9 file(s) remaining
upload: models/elmoPlotsweights50to100.pkl to s3://bdso-celebrity-content-dev-vpc-05447d97de18e28f8/models/elmoPlotsweights50to100.pkl
Completed 352.7 KiB/10.1 MiB (538.0 KiB/s) with 8 file(s) remaining
Completed 608.7 KiB/10.

## Test Elasticsearch to store the vertor and peform cosine similarity score.

In [97]:
import requests
from pprint import pprint
from datetime import datetime
from elasticsearch import Elasticsearch
import json

es= Elasticsearch([{'host':ESHOST,'port':ESPORT}])

_index='bios'
esIndex = 'http://'+ESHOST+':'+ESPORT+'/'+_index    
_doc= 'bio'

headers={"Content-Type": "application/json"}

In [56]:
request_body = {
  "mappings": {
    "properties": {
      "_actorID" : {
        "type" : "keyword"
      },  
      "celebrity" : {
        "type" : "keyword"
      },         
      "bio_vector": {
        "type": "dense_vector",
        "dims": len(xin[0])
      }

    }
  }
}

try:
    es.indices.delete(index = _index)
except:
    pass
    
es.indices.create(index = _index, body = request_body)

Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/urllib3/connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "/opt/conda/lib/python3.6/site-packages/urllib3/connectionpool.py", line 384, in _make_request
    six.raise_from(e, None)
  File "<string>", line 2, in raise_from
  File "/opt/conda/lib/python3.6/site-packages/urllib3/connectionpool.py", line 380, in _make_request
    httplib_response = conn.getresponse()
  File "/opt/conda/lib/python3.6/http/client.py", line 1331, in getresponse
    response.begin()
  File "/opt/conda/lib/python3.6/http/client.py", line 297, in begin
    version, status, reason = self._read_status()
  File "/opt/conda/lib/python3.6/http/client.py", line 266, in _read_status
    raise RemoteDisconnected("Remote end closed connection without"
http.client.RemoteDisconnected: Remote end closed connection without response

During handling of the above exception, another exception occurred:

Traceback (most recent ca

Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/urllib3/connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "/opt/conda/lib/python3.6/site-packages/urllib3/connectionpool.py", line 384, in _make_request
    six.raise_from(e, None)
  File "<string>", line 2, in raise_from
  File "/opt/conda/lib/python3.6/site-packages/urllib3/connectionpool.py", line 380, in _make_request
    httplib_response = conn.getresponse()
  File "/opt/conda/lib/python3.6/http/client.py", line 1331, in getresponse
    response.begin()
  File "/opt/conda/lib/python3.6/http/client.py", line 297, in begin
    version, status, reason = self._read_status()
  File "/opt/conda/lib/python3.6/http/client.py", line 266, in _read_status
    raise RemoteDisconnected("Remote end closed connection without"
http.client.RemoteDisconnected: Remote end closed connection without response

During handling of the above exception, another exception occurred:

Traceback (most recent ca

ConnectionError: ConnectionError(('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))) caused by: ProtocolError(('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',)))

In [57]:
ids = list(bio_dict.keys())
for i, v in zip(ids, xin):
    _id = str(i)
    #print(_id)
    doc = {
        #_actorID	actor	age	birthdate	gender	height
        "_actorID" : i,
        "celebrity" : celebrityLookup[_id],
        "bio_vector": [float(x) for x in v], 
        }
    r = requests.put(esIndex+'/_doc/'+_id, headers= headers, data = json.dumps(doc))
    print (r.text)

NameError: name 'celebrityLookup' is not defined

In [58]:
es.indices.stats(index=_index)

Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/urllib3/connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "/opt/conda/lib/python3.6/site-packages/urllib3/connectionpool.py", line 384, in _make_request
    six.raise_from(e, None)
  File "<string>", line 2, in raise_from
  File "/opt/conda/lib/python3.6/site-packages/urllib3/connectionpool.py", line 380, in _make_request
    httplib_response = conn.getresponse()
  File "/opt/conda/lib/python3.6/http/client.py", line 1331, in getresponse
    response.begin()
  File "/opt/conda/lib/python3.6/http/client.py", line 297, in begin
    version, status, reason = self._read_status()
  File "/opt/conda/lib/python3.6/http/client.py", line 266, in _read_status
    raise RemoteDisconnected("Remote end closed connection without"
http.client.RemoteDisconnected: Remote end closed connection without response

During handling of the above exception, another exception occurred:

Traceback (most recent ca

ConnectionError: ConnectionError(('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))) caused by: ProtocolError(('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',)))

In [59]:
r = requests.get(esIndex+'/_doc/1')
pprint(r.text)

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))

In [60]:
vec = json.loads(r.text)["_source"]["bio_vector"]
q={
  "query": {
    "script_score": {
      "query" : {
        "match_all" : {}
      },
      "script": {
        "source": "cosineSimilarity(params.query_vector, doc['bio_vector']) + 1.0", 
        "params": {
          "query_vector": vec
        }
      }
    }
  }
}

res= es.search(index=_index, body=q)

pprint(res)

NameError: name 'r' is not defined

In [ ]:
[hit['_source']['_actorID'] for hit in res['hits']['hits']]

# REDUCE DIMENSIONS

### Visualize the sentence arrays in reduced Dimensions using PCA and TSNE

Use PCA and t-SNE to reduce the 1,024 dimensions which are output from ELMo down to 2 so that we can review the outputs from the model.

In machine learning, dimensionality reduction is the process of reducing the number of random variables under consideration by obtaining a set of principal variables. By reducing the dimension of your feature space, you have fewer relationships between features to consider which can be explored and visualized easily and also you are less likely to overfit your model. https://www.datacamp.com/community/tutorials/introduction-t-sne

#### Principal Component Analysis or PCA 
Principal Component Analysis (PCA)is a linear feature extraction technique. It performs a linear mapping of the data to a lower-dimensional space in such a way that the variance of the data in the low-dimensional representation is maximized. It does so by calculating the eigenvectors from the covariance matrix. The eigenvectors that correspond to the largest eigenvalues (the principal components) are used to reconstruct a significant fraction of the variance of the original data.

In simpler terms, PCA combines your input features in a specific way that you can drop the least important feature while still retaining the most valuable parts of all of the features. As an added benefit, each of the new features or components created after PCA are all independent of one another.
t-Distributed Stochastic Neighbor Embedding (t-SNE)

#### t-Distributed Stochastic Neighbor Embedding (t-SNE) 
is a non-linear technique for dimensionality reduction that is particularly well suited for the visualization of high-dimensional datasets. It is extensively applied in image processing, NLP, genomic data and speech processing.


In [ ]:
### NOTE - this takes a bit of time to reduce 1024 by 15316 array into 2 dimensions
from sklearn.decomposition import PCA

pca = PCA(n_components=50)  # was 50
y = pca.fit_transform(xin)

# t-Distributed Stochastic Neighbor Embedding (t-SNE) - Dimensionality Reduction technique
from sklearn.manifold import TSNE
y = TSNE(n_components=2).fit_transform(y)

In [ ]:
# Build a unified data frame (udf) with columns: MovieID, Plot, Genre, and Y-transform
# udf = pd.DataFrame(columns=['MovieID', 'Plot', 'Genre','Y-Transform'])

i=0
row_list = []
for key in plot_dict: 
    row_dict = {'MovieID':key,
                'Plot':sentences[i],
                'Genre':genre_dict[key],
                'Y-Transform-X': y[i][0],
                'Y-Transform-Y': y[i][1],
               }
    row_list.append(row_dict)
    i += 1
udf = pd.DataFrame(row_list)

In [ ]:
udf.to_csv("UnifiedData.csv", index=False)

In [ ]:
# Test below

In [ ]:
udf.head()

In [ ]:
udf = pd.read_csv("../data/elmoWeightsAll/UnifiedData.csv", dtype={'MovieID':str} )
udf.head()